<a href="https://colab.research.google.com/github/murnanedaniel/Dynamic-Loss-Weighting/blob/master/Research/Notebooks/Video_Development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Flagging Models for MER

In [ ]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import sys
import csv

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import numpy.random as random
import seaborn as sns
import pandas as pd

# import dlib
import glob

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import cv2
import imageio
# from IPython.display import Video

In [ ]:
import wandb

ModuleNotFoundError: ignored

## Video Capture

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def getVideo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function getVideo(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Pause/Play';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // start_recording(video, capture);
      

      recordedBlobs = [];
      let options = {mimeType: 'video/webm;codecs=vp9,opus'};
      if (!MediaRecorder.isTypeSupported(options.mimeType)) {
        console.error(`${options.mimeType} is not supported`);
        options = {mimeType: 'video/webm;codecs=vp8,opus'};
        if (!MediaRecorder.isTypeSupported(options.mimeType)) {
          console.error(`${options.mimeType} is not supported`);
          options = {mimeType: 'video/webm'};
          if (!MediaRecorder.isTypeSupported(options.mimeType)) {
            console.error(`${options.mimeType} is not supported`);
            options = {mimeType: ''};
          }
        }
      }

      try {
        mediaRecorder = new MediaRecorder(video.srcObject, options);
      } catch (e) {
        console.error('Exception while creating MediaRecorder:', e);
        return;
      }

      console.log('Created MediaRecorder', mediaRecorder, 'with options', options);
      mediaRecorder.onstop = (event) => {
        console.log('Recorder stopped: ', event);
        console.log('Recorded Blobs: ', recordedBlobs);
      };
      mediaRecorder.ondataavailable = handleDataAvailable;
      mediaRecorder.start();
      console.log('MediaRecorder started', mediaRecorder);
      
      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      mediaRecorder.stop()

      // const canvas = document.createElement('canvas');
      // canvas.width = video.videoWidth;
      // canvas.height = video.videoHeight;
      // canvas.getContext('2d').drawImage(video, 0, 0);
      // stream.getVideoTracks()[0].stop();
      // div.remove();
      // return canvas.toDataURL('image/jpeg', quality);
    }
    let mediaRecorder;
    let recordedBlobs;

    function handleDataAvailable(event) {
      console.log('handleDataAvailable', event);
      if (event.data && event.data.size > 0) {
        recordedBlobs.push(event.data);
        console.log("In handle", recordedBlobs)
        // storeVideo(recordedBlobs, "test2");
        // return recordedBlobs;
      }
    }

    function getBlob() {
      console.log("In return",recordedBlobs);
      return recordedBlobs[0];
    }

    function sleep(ms) {
      return new Promise(resolve => setTimeout(resolve, ms));
    }

    // function start_recording(vid, capture) {
    //   recordedBlobs = [];
    //   let options = {mimeType: 'video/webm;codecs=vp9,opus'};
    //   if (!MediaRecorder.isTypeSupported(options.mimeType)) {
    //     console.error(`${options.mimeType} is not supported`);
    //     options = {mimeType: 'video/webm;codecs=vp8,opus'};
    //     if (!MediaRecorder.isTypeSupported(options.mimeType)) {
    //       console.error(`${options.mimeType} is not supported`);
    //       options = {mimeType: 'video/webm'};
    //       if (!MediaRecorder.isTypeSupported(options.mimeType)) {
    //         console.error(`${options.mimeType} is not supported`);
    //         options = {mimeType: ''};
    //       }
    //     }
    //   }

    //   try {
    //     mediaRecorder = new MediaRecorder(vid.srcObject, options);
    //   } catch (e) {
    //     console.error('Exception while creating MediaRecorder:', e);
    //     return;
    //   }

    //   console.log('Created MediaRecorder', mediaRecorder, 'with options', options);
    //   mediaRecorder.onstop = (event) => {
    //     console.log('Recorder stopped: ', event);
    //     console.log('Recorded Blobs: ', recordedBlobs);
    //   };
    //   mediaRecorder.ondataavailable = handleDataAvailable;
    //   mediaRecorder.start();
    //   console.log('MediaRecorder started', mediaRecorder);
    //   // await new Promise((resolve) => capture.onclick = resolve);
      
    //   setTimeout(mediaRecorder.stop(), 3000);

    //   return recordedBlobs;
    // }

    ''')
  display(js)
  _ = eval_js('getVideo({})'.format(quality))
  data = eval_js('getBlob()')
  # print(data)
  binary = b64decode(data)
  print(binary)
  # with open(filename, 'wb') as f:
  #   f.write(binary)
  return data

In [ ]:
from IPython.display import Image
try:
  data = getVideo()
  # print('Saved to {}'.format(filename))
  # print(data)
  # Show the image which was just taken.
  # display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

<IPython.core.display.Javascript object>

argument should be a bytes-like object or ASCII string, not 'dict'


In [ ]:
data[0]

{}

Previous structure

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Pause/Play';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      // const canvas = document.createElement('canvas');
      // canvas.width = video.videoWidth;
      // canvas.height = video.videoHeight;
      // canvas.getContext('2d').drawImage(video, 0, 0);
      // stream.getVideoTracks()[0].stop();
      // div.remove();
      // return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return data

In [ ]:
from IPython.display import Image
try:
  data = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
Test